---
title: "Comparative Metagenomics"
author:
  - name: Sam Hillman
    affiliation:
      - id: mia
        name: Department of Computing, University of Turku, Finland
toc: true
number-sections: true
categories: [R]
execute: 
  eval: true
---

::: {style="max-width:1200px"}
![](./../_resources/mgnify_logo.png)

# Comparative metagenomics
# Normalization methods, alpha & beta diversity, and differentially abundant taxa

In this notebook we aim to demonstrate how the MGnifyR tool can be used to fetch 
data and metadata of a MGnify metagenomic analysis 

Then we show how to generate diversity metrics for comparative metagenomics using
taxonomic profiles. 

Finally, we use the `mikropml` package to detect differentially abundant taxa and
to build an ML classification model. 

It's important to note, we're using methods and protocols that align with the 
[mia](https://microbiome.github.io/mia/) package, which can be read about further 
in the [OMA](https://microbiome.github.io/OMA/docs/devel/) guides.

[MGnifyR](https://www.bioconductor.org/packages/release/bioc/html/MGnifyR.html) 
is a library that provides a set of tools for easily accessing and processing 
MGnify data in R, making queries to MGnify databases through the [MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1/). 

The benefit of MGnifyR is that data can either be fetched in tsv format or be directly 
combined in a [TreeSummarizedExperiment](https://www.bioconductor.org/packages/release/bioc/vignettes/TreeSummarizedExperiment/inst/doc/Introduction_to_treeSummarizedExperiment.html)
object to run an analysis in a custom workflow.

`TreeSummarizedExperiment` is supported by the [mia bioconductor package](https://microbiome.github.io/mia/), 
which is dedicated to microbiome analysis.

This is an interactive code notebook (a Jupyter Notebook). To run this code, 
click into each cell and press the ▶ button in the top toolbar, or press `shift+enter`.

------------------------------------------------------------------------

```{r}
# Loading libraries
suppressMessages({
  library(ggplot2)
  library(MGnifyR)
  library(tidyverse)
  library(vegan)
  library(mia)
  library(miaViz)
  library(biomformat)
  library(scater)
  library(metagenomeSeq)
  library(reshape2)
  library(viridis)
  library(patchwork)
  library(IRdisplay)
  library(ALDEx2)
  library(ANCOMBC)
  library(mikropml)
  library(MLeval)
  library(caret)
  library(scales)
})

display_markdown(file = '../_resources/mgnifyr_help.md')
```

# Fetching data and preprocessing
In this example we are going to fetch MGnify analysis results and metadata for a 
dataset on the human gut microbiome viewed across age and geography - 
([MGYS00005154](https://www.ebi.ac.uk/metagenomics/studies/MGYS00005154)).

## Setting the study accession
1. First, we need to specify the accession number of the study we're working with. 
This can be done by setting the `mgnify_study_accession` variable. The accession 
number uniquely identifies the study in the MGnify database.

```{r}
#| output: false

source("./utils/variable_utils.r")

mgnify_study_accession <- get_variable_from_link_or_input('MGYS', 'Study Accession', 
                                                          'MGYS00005154')

# You can also just directly set the accession variable in code, like this:
# mgnify_study_accession <- "MGYS00005292"
```

2. To interact with the MGnify database, we need to create an MgnifyClient object. 
This object allows us to fetch data from MGnify, and we can configure it to use 
a cache for efficiency. 

```{r}
#| output: false
# Check if the cache directory exists, if not, create it
if (!dir.exists("./.mgnify_cache")) {
  dir.create("./.mgnify_cache", recursive = TRUE)
}

# Create the MgnifyClient object with caching enabled
mg <- MgnifyClient(usecache = TRUE, cache_dir = "./.mgnify_cache")
```

3. Using the MgnifyClient object, we can search for all analyses associated with 
the study accession number we set earlier. This will return a list of analysis 
accession numbers.

```{r}
#| output: false
analyses_accessions <- searchAnalysis(mg, "studies", mgnify_study_accession)
analyses_accessions
```

## Download metadata
1. First, download the metadata for the first 20 Analyses and put it into a dataframe.

```{r}
#| output: false
analyses_metadata_df <- getMetadata(mg, head(analyses_accessions, 30))
```

2. The table could be big, so let's look at a sample of it (`head`).

```{r}
#| output: false
t(head(analyses_metadata_df))
```

## Convert to `TreeSummarizedExperiment` object.

```{r}
#| output: false
tse <- getResult(mg, analyses_metadata_df$analysis_accession)
```

# Normalization, alpha diversity indices and taxonomic profiles visualization

Let's Show some stats on sequencing depth across samples.

```{r}
# Calculate and print max difference
max_difference <- max(tse$sum) / min(tse$sum)
sprintf("The max difference in sequencing depth is %s", max_difference)
```

An approximate 4.5-fold difference in the library sizes means that we will need 
to apply a normalization method before continuing with the analysis. The most 
common normalization methods used in microbiome count data are proportions and rarefaction. 

However, other methods originally developed to normalize RNA-seq counts have been 
adapted to differential-abundance analysis in microbiome data. A discussion about 
how to choose the right normalization method is out of the scope of this material, 
but the topic has been covered in multiple forums and scientific publications. 

Depending on the downstream analysis we intend to perform, different methods might 
be appropriate. For instance, to compare groups of samples at community-level 
through beta-diversity, “…proportions and rarefying produced more accurate comparisons 
among communities and are the only methods that fully normalized read depths across 
samples.

Additionally, upper quartile, cumulative sum scaling (CSS), edgeR-TMM, and DESeq-VS 
often masked differences among communities when common OTUs differed, and they 
produced false positives when rare OTUs differed” ([1](#ref-mcknight2019)). On 
the other hand, for detection of differentially abundant species, “both proportions 
and rarefied counts result in a high rate of false positives in tests for species 
that are differentially abundant across sample classes” ([2](#ref-mcmurdie2014)).

However, rarefaction can utilize information in the full read count data, and 
the debate on the pros and cons of rarefaction is ongoing ([3](#ref-schloss2024a), 
[4](#ref-schloss2024b)).

In the following examples we will show two popular ways of normalization: 
relative abundance transformation (transforming features into zero mean, unit variance - 
also called relative abundance transformation) and rarefaction.

## Normalization by total sum scaling (TSS, relative abundance or proportions)
The simplest way to normalize the differences in sample size is to do a relative 
abundance transformation on the OTU counts table by dividing by the number of 
total reads of each sample. This type of normalization is also referred to as 
relative abundance or proportions. 

We use this normalization to compare taxonomic profiles, while alpha diversity 
indices are computed on the non-normalized matrix. The reason to do so is that 
we need a matrix of integer numbers as input.

1. Apply the relative abundance transformation to raw counts matrix.

```{r}
tse <- transformAssay(tse, name = "relabundance", method = "relabundance")
```

2. Agglomerate taxonomy at Class rank and save to the `glom_tse_relab` object 
within `altExp`. Then remove all assays other than `relabundance` for simplicity.

```{r}
# Agglomerate at class rank
altExp(tse, "glom_tse_relab") <- agglomerateByRank(tse, rank = "Class")

# Remove assays other than relabundance
assays(altExp(tse, "glom_tse_relab")) <- assays(altExp(tse, "glom_tse_relab"))["relabundance"]
```

3. Get the top 15 features of the `relabundance` assay and subset the `altExp` 
`relabundance` assay by those top 15 features.

```{r}
# Take the top 15 features based on their means
top15_relabundance <- getTop(altExp(tse, "glom_tse_relab"), method = "mean", 
                             assay.type = "relabundance", top = 15L)

# Subset relabundance tse by top features
altExp(tse, "glom_tse_relab") <- altExp(tse, "glom_tse_relab")[top15_tss, ]
```

## Normalization by subsampling (rarefaction)
Rarefaction is an alternative to doing a relative abundance transformation to 
obtain an adjusted OTUs count matrix. The method is based on a process of subsampling 
to the smallest library size in the data set. 

The algorithm randomly removes reads until the samples reach the same library size. 
Despite the apparent disadvantage of discarding information from the larger samples, 
rarefaction is quite popular in microbial ecology. 

The first step is to find the smallest sample size. We can use the number of 
observed OTUs in the original matrix to do so.

1. Find the smallest sample size.

```{r}
min_size <- min(colSums2(assay(tse, "counts")))
```

2. Rarefying to the smallest sample.

```{r}
tse <- rarefyAssay(tse, assay.type = "counts", size = min_size, name = "rarefy")
```

3. Agglomerate taxonomy at Class rank and save to the `glom_tse` object within `altExp`. 
Then remove all assays other than `rarefy` for simplicity.

```{r}
# Agglomerate at class rank
altExp(tse, "glom_tse_rarefy") <- agglomerateByRank(tse, rank = "Class")

# Remove assays other than rarefy
assays(altExp(tse, "glom_tse_rarefy")) <- assays(altExp(tse, "glom_tse_rarefy"))["rarefy"]
```


4. Get the top 15 features of the `rarefy` assay and subset the `altExp`
`rarefy` assay by those top 15 features. 

```{r}
# Get the top features
top15_rarefy <- getTop(altExp(tse, "glom_tse_rarefy"), top = 15L, 
                       assay.type = "rarefy", method = "mean")

# Subset original rarefy tse by top features
altExp(tse, "glom_tse_rarefy") <- altExp(tse, "glom_tse_rarefy")[top15_rarefy, ]
```

## OTU Filtering

1. First, identify the total counts per sample and add them to the colData of our 
`tse` object.

```{r}
tse <- addPerCellQC(tse)
```

2. We then plot a histogram to identify outliers, so we can remove samples with 
low coverage.

```{r}
scatter_plot <- plotColData(tse, y = "sum") +
   scale_y_log10(
    breaks = trans_breaks("log10", function(x) 10^x, n = 8),
    labels = trans_format("log10", math_format(10^.x))
  ) +
  labs(x = "Samples", y = "Library size (log scale)") 
```

3. We can see that samples with number of reads above 10^6.2 seem to be outliers. 
Let’s filter out the outliers and plot a new histogram.

At the same time, we can also filter out singletons. Singletons are OTUs of size 
one, meaning that only one read was assigned to that OTU. These very low-abundance 
OTUs could be biologically real (belonging to the rare biosphere ([5](#ref-lynch2015))), 
or they could be false positives due to sequencing artefacts. Singletons observed 
in only one sample are more likely to be artefacts, and it is good practice to 
remove them from the OTUs counts table to avoid artificially over-estimating the 
OTUs richness. You can find more discussion about this in [Robert Edgar’s blog](https://drive5.com/usearch/manual/singletons.html).

```{r}
# Subset the tse object to keep only samples with library sizes > 10^6.2 
# and non-singleton taxa in the filtered data
tse <- tse[rowSums(assay(tse, "counts")) > 1, tse$sum < 10^6.2]

# Now we redo the scatter plot
filtered_scatter_plot <- plotColData(tse, y = "sum") +
   scale_y_log10(
    breaks = trans_breaks("log10", function(x) 10^x, n = 8),
    labels = trans_format("log10", math_format(10^.x))
  ) +
  labs(x = "Samples", y = "Library size (log scale)") 
```

## Computing alpha diversity indices
Alpha diversity is a measure of species diversity in a particular area or an ecosystem. 
In terms of microbial ecology, analyzing the sample diversity of sequencing data 
is commonly the first step in assessing differences between microbial environments. 
Some of the most popular alpha diversity indices reported in microbial community 
analyses are Chao, abundance-based coverage estimators (ACE), Simpson, and Shannon-Weaver.

Chao1 and ACE are nonparametric estimators that describe diversity as the max 
number of expected species in the sample (species richness). They consider the 
proportion of species that have been observed before (“recaptured”) to those that 
are observed only once. Both Chao1 and ACE underestimate true richness at low sample sizes.

On the other hand, Simpson and Shannon-Weaver, consider relative abundances and 
depends on both, species richness and the evenness (or equitableness), with which 
individuals are distributed among the different species. However, both metrics have 
specific biases. The Shannon-Weaver index places a greater weight on species richness, 
whereas the Simpson index considers evenness more than richness in its measurement.

More discussion and examples illustrating how sample size and normalization methods 
can affect alpha diversity metrics in references [6](#ref-willis2019) and [7](#ref-kim2017).

In this example, we're going to calculate the diversity measures for our original 
counts data, and for our rarefied original counts data.

1. Now, let's use the `addAlpha` method to compute the alpha diversity indices 
on original raw counts data, and we use the `niter` argument to rarefy our original 
counts data to calculate the shannon diversity for rarefied data. We also add a 
`SampleID` colData column to `tse` to help with out plotting too. 

For this example we'll use the `shannon_diversity` index to show alpha diversity, 
though [see here](https://microbiome.github.io/mia/reference/addAlpha.html) for 
more index options.

```{r}
# Add SampleID to colData
tse$SampleID <- colnames(tse)

# Compute alpha diversity on normal counts data
tse <- addAlpha(tse, index = "shannon_diversity", name = "shannon_diversity_normal")

# Compute alpha diversity on rarefied data
tse <- addAlpha(tse, index = "shannon_diversity", name = "shannon_diversity_rarefy", 
                niter = 10)
```

2. Plot alpha diversity with `plotColData`, generating a plot for our shannon 
diversity calculation on both our original counts data and rarefied data. We save 
both plots to `plots`.

```{r}
# Use lapply to generate plots
plots <- lapply(c("shannon_diversity_normal", "shannon_diversity_rarefy"), function(y) {
  label <- ifelse(y == "shannon_diversity_normal", "normal", "rarefy")
  plotColData(tse, x = "SampleID", y = y, colour_by = "SampleID") +
    labs(y = paste("Shannon Diversity Index (", label, ")", sep = "")) +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 6))
})
```

3. Combine all plots using patchwork.

```{r}
# Combine all plots using patchwork
wrapped_plots <- wrap_plots(plots) +
    plot_layout(guides = "collect") + 
    plot_annotation(title = "Shannon Diversity Calculations",
                    theme = theme(plot.title = element_text(size = 12, hjust = 0.5)))
```

## Community profile visualization
Here we visualise the taxonomic profiles in barplots at class ranks for our top 
tss normalized taxa and rarefied taxa.

1. Let's store our `glom_tse_relab` and `glom_tse_rarefy` objects in 
variables to make referencing them easier.

```{r}
glom_tse_relab <- altExp(tse, "glom_tse_relab")
glom_tse_rarefy <- altExp(tse, "glom_tse_rarefy")
```


2. Lets change any unknown taxonomic rowData information to "Unknown" from "NA" 
in our `tse` objects.

```{r}
rowData(glom_tse_relab)$Class <- ifelse(is.na(rowData(glom_tse_relab)$Class), "Unknown", 
                                        rowData(glom_tse_relab)$Class)
rowData(glom_tse_rarefy)$Class <- ifelse(is.na(rowData(glom_tse_rarefy)$Class), "Unknown", 
                                         rowData(glom_tse_rarefy)$Class)
```

3. Create the plots with the `plotAbundance` `mia` method. We use our agglomerated 
`relabundance` and `rarefy` assays from our `tse` objects to do this.

```{r}
# Plotting our tss data
relabundance_plot <- plotAbundance(glom_tse_relab,
                          assay.type = "relabundance",
                          rank = "Class",
                          order_rank_by = "abund")

# Plotting our rarefied data
rarefy_plot <- plotAbundance(glom_tse_rarefy,
                             assay.type = "rarefy",
                             rank = "Class",
                             order_rank_by = "abund")
```

By naked eye, we can see that the profiles change. Let’s take a deeper look at 
some interesting Classes to check how the abundance change depending on the 
normalization method.

1. Define our subset classes

```{r}
subset_classes <- c("Betaproteobacteria", 
                    "Bacteroidia", 
                    "Actinobacteria")
```


2. Subset our `tse` objects.

```{r}
glom_tse_relab <- glom_tse_relab[rowData(glom_tse_relab)$Class %in% subset_classes, ]
glom_tse_rarefy <- glom_tse_rarefy[rowData(glom_tse_rarefy)$Class %in% subset_classes, ]
```

3. Plot our new plots with the `plotAbundance` `mia` method.

```{r}
relabundance_plot <- plotAbundance(glom_tse_relab, 
                                   assay.type = "relabundance", 
                                   rank = "Class", 
                                   order_rank_by = "abund")

rarefy_plot <- plotAbundance(glom_tse_rarefy, 
                             assay.type = "rarefy", 
                             rank = "Class", 
                             order_rank_by = "abund")
```

## Comparative metagenomics at community level: Beta diversity

Beta (β)-diversity is used in ecology to assess the heterogeneity of local communities. 
It is defined as the ratio between regional and local species diversity. In simpler 
terms, it calculates the number of species that are not the same in two different 
environments.

Some of the most popular beta diversity measures in microbiome research include 
Bray-Curtis index (for compositional data), Jaccard index (for presence/absence 
data, ignoring abundance information), Aitchison distance (Euclidean distance 
for transformed abundances, aiming to avoid the compositionality bias), and the 
Unifrac distances (that use the phylogenetic tree information). 

Only some of the commonly used beta diversity measures are actual distances, 
therefore, the term dissimilarity or beta-diversity is commonly used. The workflow 
to assess β-diversity between groups of samples starts with calculating the diversity 
indices for each pair of samples. This results in a distance (or dissimilarity) 
matrix that is often used for ordination (e.g., principal coordinates analysis) 
and data exploration in microbiota data analysis ([8](#ref-wagner2018)).

For statistical analysis, [adonis](https://www.rdocumentation.org/packages/vegan/versions/2.6-4/topics/adonis) 
and [betadisper](https://www.rdocumentation.org/packages/vegan/versions/2.6-4/topics/betadisper) 
methods are widely used together in β-diversity analyses.

PERMANOVA tests whether two or more groups have different compositions. PERMANOVA 
calculates the squared deviations of each site to the centroid and then, performs 
significance tests using F-tests based on sequential sums of squares from 
permutations of the raw data. It can be seen as an ANOVA using distance matrices 
(analogous to MANOVA). A non significant p-value (p>0.05), means that there’s no 
significant difference in composition between groups (we accept the null hypothesis 
of no significant difference in composition between groups).

Betadisper tests if two or more groups are homogeneously dispersed in relation to 
their species in studied samples. This test can be done to see if one group has 
more compositional variance than another. The method use the distance matrix to 
calculate the multivariate dispersions (variances; average distance to centroids). 

Then we use group dispersions to perform an ANOVA test. ANOVA’s p-value not significant 
(p>0.05) means that group dispersions are homogenous (null hypothesis of no difference 
in dispersion between groups).

Compute beta diversity using various methods to calculate distance, and perform 
principle components analysis ploting the first two axes. According to Pereira 
et al., (2018)([9](#ref-pereira2018)), the best normalization method for metagenomic 
gene abundance is CSS for large group sizes. For this reason, we will use this 
method to show beta-diversity. 

Using as a guide the steps described in ([10](#ref-mcmurdie2018)), we will create 
a list of suitable distance methods, iterate through them, and display a combined 
plot. For a better visualization we are going to show the 95% confidence region 
with an ellipse.

1. Let's define the list of methods we're going to test.

```{r}
# Define method list
vegdist_methods <- c("manhattan", "euclidean", "bray", "jaccard",
                     "gower", "chao")

# Create empty vectors to hold method names and plots
created <- c()
plots <- c()
```

2. Loop through these method types and plot abundance for each, saving each plot 
into the `plots` vector.

```{r}
# Loop through these methods and plot abundance
for (method in vegdist_methods) {
  # Set the method name
  method_name <- paste("MDS", method, sep = "_")
  created <- method_name
  
    # Run MDS 
    mds_tse <- runMDS(tse,
                  FUN = vegan::vegdist,
                  method = method,
                  assay.type = "relabundance",
                  name = method_name)
  
  # Create plot object of reducedDim slot
  p <- plotReducedDim(mds_tse, dimred = paste("MDS", method, sep = "_"))
  
  # Calculate explained variance
  e <- attr(reducedDim(mds_tse, method_name), "eig")
  rel_eig <- e / sum(e[e > 0])
  
  # Add explained variance for each axis
  p <- p + labs(x = paste("MDS_1_", method, " (", round(100 * rel_eig[1], 1), "%)", 
                          sep = ""),
                y = paste("MDS_2_", method, " (", round(100 * rel_eig[2], 1), "%)", 
                          sep = ""),
                title = paste("MDS using distance method", method)) +
    theme_minimal(base_size = 6) + # Use a minimal theme for clarity
    theme(
      plot.title = element_text(hjust = 0.5), # Center the title
      axis.title = element_text(size = 6), # Smaller axis titles
      axis.text = element_text(size = 6), # Smaller axis labels
      legend.position = "bottom", # Position the legend at the bottom
      legend.title = element_text(size = 6), # Smaller legend title
      legend.text = element_text(size = 6) # Smaller legend text
    )
  
  # Save plot
  plots[[method_name]] <- p
}
```

3. Combine all plots using patchwork.

```{r}
# Combine all plots using patchwork
wrapped_plots <- wrap_plots(plots) +
  plot_layout(guides = "collect") + 
  plot_annotation(title = "MDS on Various Distance Metrics",
                  theme = theme(plot.title = element_text(size = 14, hjust = 0.5)))
```

Separation of the groups observed in some of the ordination plots could be due to 
a location effect, dispersion effect, or both. To test the location effect (null 
hypothesis of no difference in composition between groups) we use a permanova test, 
using the bray method to calculate the distance matrix. 

PERMANOVA assumes there is homogeneity of dispersion among groups. We want to also 
test this assumption, to check whether the differences detected by our permanova 
test are due to variation in dispersion of the data. 

We do this using the betadisper method to calculate variances and then evaluate 
if there’s a significant variation between groups through an anova.

To do all these tests, we use the `addRDA` method, which will do all of them for us.

```{r}
beta_diversity_tse <- addRDA(
  tse,
  formula = data ~ sample_geographic.location..country.and.or.sea.region.,
  assay.type = "relabundance",
  method = "bray",
  test.signif = TRUE,
  homogeneity.test = "anova",
  full = TRUE
)
```

We now plot and fetch the results

```{r}
# Plot the results
plotReducedDim(beta_diversity_tse, "RDA", 
               colour_by = "sample_geographic.location..country.and.or.sea.region.")

# Fetch significance results, including PERMANOVA and betadisper (homogeneity) results
significance_results <- attr(reducedDim(beta_diversity_tse, "RDA"), "significance")
significance_results
```

ANOVA's p-value is not significant (p>0.05), meaning that there's no significant 
difference in dispersion among groups. The PERMANOVA however showed a significant 
P value (p<0.05), meaning significant differences in community composition between 
the groups. 

## Detection of differentially abundant taxa
There are many approaches out there to detect differentially abundant taxa (or genes).
Here we detect microbiome community composition changes due to various environmental 
factors. Describing such associations in quantitative models has multiple applications, 
such as in predicting disease status based on microbiome data. 

In addition, the pipeline includes the generation of high-quality plots for visual 
inspection of results ([11](#ref-wirbel2021)). There are multiple parameters that 
can be adjusted for better performance in each step of the analysis. However, in 
this exercise we are using default arguments with minimal tuning.

### Association testing

1. First we use our original counts data on our `tse` object and agglomerate 
by rank to a new `genus_tse` object.

```{r}
genus_tse <- agglomerateByRank(tse, rank = "Genus")
```

2. Subset the dataset by prevalence to include only the taxa that are present in 
at least 25% of the samples.

```{r}
genus_tse <- subsetByPrevalent(genus_tse, prevalence = 0.25, include.lowest = TRUE)
```

3. Subset our `genus_tse` samples into 2 geographic groups, and add an extra `colData` 
column to accommodate this.

```{r}
# Create a new condition vector
conditions <- ifelse(genus_tse$sample_geographic.location..country.and.or.sea.region. 
                     == "GAZ:United States of America", "USA", "Other")

# Add conditions as a new column in the colData of genus_tse
genus_tse$geo_location <- conditions
```

4. Now we use the `ALDEx2` package to compute the differential abundance analysis.

```{r}
# Set seed for reproducibility
set.seed(123)

# Generate Monte Carlo samples of the Dirichlet distribution for each sample using 
# the geographic location for grouping
x <- aldex.clr(assay(genus_tse, "counts"), genus_tse$geo_location)

# Calculates expected values of the Welch's t-test and Wilcoxon rank
x_tt <- aldex.ttest(x, paired.test = FALSE, verbose = FALSE)

# Determine the median clr abundance of the feature in all samples and in groups, 
# the median difference between the two groups, the median variation within each 
# group and the effect size - which is the median of the ration of the between 
# group difference and the larger of the variance within groups
x_effect <- aldex.effect(x, CI = TRUE, verbose = FALSE)

# Now combine all outputs into a data frame
aldex_out <- data.frame(x_tt, x_effect)

# Now we plot a Bland-ALtman plot (left) and a dispersion plot (right)
par(mfrow = c(1, 2))

# Set up a 1x2 plotting area
par(mfrow = c(1, 2))

# Bland-Altman plot (left)
aldex.plot(aldex_out,
  type = "MA",
  test = "welch",
  xlab = "Log-ratio abundance",
  ylab = "Difference",
  cutoff = 0.05)

# Dispersion plot (right)
aldex.plot(aldex_out,
  type = "MW",
  test = "welch",
  xlab = "Dispersion",
  ylab = "Difference",
  cutoff = 0.05)

# Reset plotting layout
par(mfrow = c(1, 1))

# Then do some further evaluation on the results
aldex_out %>%
  rownames_to_column(var = "Genus") %>%
  # here we choose the wilcoxon output rather than t-test output
  filter(wi.eBH <= 0.05)  %>%
  dplyr::select(Genus, we.eBH, wi.eBH, effect, overlap) %>%
  knitr::kable()
```

## Confounder Testing 

Test the associated metadata variables for potential confounding influence. As many 
biological and technical factors beyond the primary phenotype of interest can influence
microbiome composition, simple association studies may suffer confounding by other 
variables, which can lead to spurious results. 

Associations are visualized either as barplot or Q-Q plot, depending on the type 
of metadata. Additionally, it evaluates associations among metadata variables 
using conditional entropy and associations with the label using generalized linear 
models, producing a correlation heatmap and appropriate quantitative barplots, 
respectively.

Here we use the `ANCOMBC` package to test for cofounders.

1. Compute and store the library size in `colData`.

```{r}
genus_tse$library_size <- colSums(assay(genus_tse, "counts"))
```

2. The `ancombc2_out` method performs the differential abundance analysis whilst 
accounting for multiple covariates and potential confounders. 

```{r}
# Perform the analysis 
ancombc2_out <- ancombc2(genus_tse,
                         assay.type = "counts",
                         fix_formula = "sample_geographic.location..country.and.or.sea.region. + library_size",
                         p_adj_method = "fdr",
                         lib_cut = 0,
                         group = "sample_geographic.location..country.and.or.sea.region.", 
                         struc_zero = TRUE, 
                         neg_lb = TRUE,
                         alpha = 0.05,
                         global = TRUE)
```

## Generation of a predictive model: Machine learning workflow

We'll now use the `mikropml` library to construct a machine learning meta-analysis
models on the basis of microbial markers. The steps to follow include functions 
for data normalization, splitting the data into cross-validation folds, training 
the model, and making predictions based on cross-validation instances and the trained 
models. 

Such a model can be built on one dataset and then be applied on another, similarly 
processed holdout dataset. This might be of interest when comparing data from two 
different studies testing the same condition.

We will work on our original full counts `tse` object.

1. Normalize the data using CLR transformation.

```{r}
tse <- transformAssay(tse, method = "clr", MARGIN = "samples", pseudocount = 1)
assay <- assay(tse, "clr")
```

2. Prepare the cross-validation data frame. Cross-validation is a technique to 
assess how well a machine learning model would generalize to external data by 
partitioning the dataset into training and test sets. Here, we prepare the data 
frame needed for cross-validation.

```{r}
# Prepare the cross-validation data frame.
# Transpose the assay matrix to have samples as rows and features (taxa) as columns
assay <- t(assay)

# Convert the transposed matrix to a data frame for easier manipulation and analysis
df <- as.data.frame(assay)

# Extract the geographic location labels from the colData of the tse object
labels <- colData(tse)$sample_geographic.location..country.and.or.sea.region.

# Simplify the labels to make them valid R variable names
labels <- gsub(" ", "_", labels)
labels <- gsub("[:.]", "", labels)

# Convert the geographic location labels to a factor (categorical variable)
labels <- as.factor(labels)

# Add the geographic location labels as a new column in the data frame
df$geo_location <- labels
```

3. Train the model using `mikropml`. Model training involves selecting and applying 
a machine learning algorithm to the data. Here, we use the random forest algorithm 
(rf) to train our model. We do 10 folds for our cross-validation, and repeat the 
process 5 times.

```{r}
results <- run_ml(df, "rf", outcome_colname = "geo_location", kfold = 10, 
                  cv_times = 5, training_frac = 0.8)
```

4. Model evaluation involves assessing the performance of the trained model. 
Here, we use a confusion matrix to evaluate the model's predictions. 

A confusion matrix provides a summary of prediction results on a classification 
problem, showing the number of correct and incorrect predictions made by the model 
compared to the actual outcomes.

```{r}
conf_matrix <- confusionMatrix(data = results$trained_model$finalModel$predicted, 
                               reference = results$trained_model$finalModel$y)
```

5. Evaluate model using `MLEval`. This generates detailed evaluation metrics.

```{r}
res <- evalm(results$trained_model, showplots = FALSE)
```

6. Plot ROC and precision-recall curves side-by-side. Visualization is an important 
step to understand the performance of the model. Here, we plot the ROC and precision-recall 
curves.

```{r}
roc_pr_plot <- res$roc + res$proc + plot_layout(guides = "collect") & theme(legend.position = 'bottom')
```

# References

1. <a name="ref-mcknight2019"></a>McKnight, DT., Huerlimann, R., Bower, DS. et al. 
Methods for Normalizing Microbiome Data: An Ecological Perspective. Methods in 
Ecology and Evolution / British Ecological Society 10(3):389–400 (2019) 
[DOI:10.1111/2041-210X.13115](https://doi.org/10.1111/2041-210X.13115)

2. <a name="ref-mcmurdie2014"></a>McMurdie, PJ., Holmes, S. Waste Not, Want Not: 
Why Rarefying Microbiome Data Is Inadmissible. PLoS Comput Biol 10(4):e1003531 (2014) 
[DOI:10.1371/journal.pcbi.1003531](https://doi.org/10.1371/journal.pcbi.1003531)

3. <a name="ref-schloss2024a"></a>Schloss, Patrick D. ‘Waste Not, Want Not: 
Revisiting the Analysis That Called into Question the Practice of Rarefaction’. 
mSphere 9, no. 1 (2024): e00355-23 [DOI:10.1128/msphere.00355-23](https://journals.asm.org/doi/10.1128/msphere.00355-23)

4. <a name="ref-schloss2024b"></a>Schloss PD. 2024. Rarefaction is currently the 
best approach to control for uneven sequencing effort in amplicon sequence analyses. 
mSphere 9 [DOI:10.1128/msphere.00354-23](https://journals.asm.org/doi/10.1128/msphere.00354-23)

5. <a name="ref-lynch2015"></a>Lynch, M., Neufeld, J. Ecology and exploration of 
the rare biosphere. Nat Rev Microbiol 13:217–229 (2015) [DOI:10.1038/nrmicro3400](https://doi.org/10.1038/nrmicro3400)

6. <a name="ref-willis2019"></a>Willis, A. Rarefaction, Alpha Diversity, and Statistics. 
Front. Microbiol. Sec. Terrestrial Microbiology 10:2407 (2019) [DOI:10.3389/fmicb.2019.02407](https://doi.org/10.3389/fmicb.2019.02407)

7. <a name="ref-kim2017"></a>Kim, BR., Shin, J., Guevarra, R. et al. Deciphering 
Diversity Indices for a Better Understanding of Microbial Communities. J Microbiol 
Biotechnol 28;27(12):2089-2093 (2017) [DOI:10.4014/jmb.1709.09027](https://doi.org/10.4014/jmb.1709.09027)

8. <a name="ref-wagner2018"></a>Wagner, BD., Grunwald, GK., Zerbe, GO. et al. On 
the Use of Diversity Measures in Longitudinal Sequencing Studies of Microbial 
Communities. Front. Microbiol. 9:1037 (2018) [DOI:10.3389/fmicb.2018.01037](https://doi.org/10.3389/fmicb.2018.01037)

9. <a name="ref-pereira2018"></a>Pereira, M., Wallroth, M., Jonsson, V. et al. 
Comparison of normalization methods for the analysis of metagenomic gene abundance 
data. BMC Genomics 19,274 (2018) [DOI:10.1186/s12864-018-4637-6](https://doi.org/10.1186/s12864-018-4637-6)

10. <a name="ref-mcmurdie2018"></a>McMurdie, P., Holmes, S. Tutorial of 2018: The 
distance function in phyloseq. Available at website

11. <a name="ref-wirbel2021"></a>Wirbel, J., Zych, K., Essex, M. et al. Microbiome 
meta-analysis and cross-disease comparison enabled by the SIAMCAT machine learning 
toolbox. Genome Biol 22:93 (2021) [DOI:10.1186/s13059-021-02306-1](https://doi.org/10.1186/s13059-021-02306-1)